<a href="https://colab.research.google.com/github/jayashalakshani/Real-Time-Social-Media-Sentiment-and-Trend-Analysis-Platform/blob/main/Save_Mastodon_Trend_%2B_Post_Data_to_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymongo[srv] mastodon.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.0 MB/s eta 0:00:00


# MongoDB Setup

In [2]:
from pymongo import MongoClient

# Replace these with your credentials
username = "colab_user"
password = "nqAJYTXqcLsxAoHi"
cluster_url = "cluster0.8ad48r1.mongodb.net"

In [3]:
# Full URI
uri = f"mongodb+srv://{username}:{password}@{cluster_url}/?retryWrites=true&w=majority&appName=Cluster0"

In [4]:
# Connect to MongoDB Atlas
client = MongoClient(uri)

In [5]:
# check available databases
client.list_database_names()

['social_media_analytics', 'admin', 'local']

In [6]:
# Create / select a database and collection
db = client["social_media_analytics"]

In [7]:
# Two collections
sentiment_collection = db["sentiment_data"]
trend_collection = db["trend_data"]

In [8]:
# check the available collection
db.list_collection_names()

['trend_collection', 'sentiment_data']

# Get trending tags at Mastodon

In [9]:
from mastodon import Mastodon
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize Mastodon API
mastodon = Mastodon(
    access_token="FaX6w1ZcwirKyoI_5f_IV6OA-4hM9Adk18VMFu68Zs8",
    api_base_url="https://mastodon.social"
)

# Function to extract trending tags
def extract_trending_tags():
    """Extract trending hashtags from Mastodon"""
    print("Extracting trending hashtags from Mastodon...")

    try:
        # Get trending tags
        trends = mastodon.trending_tags()

        # Create DataFrame for trending tags
        trend_data = []

        for trend in trends:
            trend_info = {
                'tag': trend['name'],
                'url': trend['url'],
                'history': trend.get('history', []),
                'uses_today': 0,
                'accounts_today': 0,
                'uses_week': 0,
                'total_score': 0
            }

            # Process history data if available
            if trend_info['history']:
                for day in trend_info['history']:
                    if 'uses' in day:
                        trend_info['uses_week'] += int(day['uses'])
                    if 'accounts' in day and day['day'] == '0':
                        trend_info['accounts_today'] = int(day['accounts'])
                    if 'uses' in day and day['day'] == '0':
                        trend_info['uses_today'] = int(day['uses'])

                # Calculate a score based on recency and popularity
                trend_info['total_score'] = trend_info['uses_today'] * 3 + trend_info['uses_week']

            trend_data.append(trend_info)

        trends_df = pd.DataFrame(trend_data)
        print(f"Extracted {len(trends_df)} trending tags")

        return trends_df

    except Exception as e:
        print(f"Error extracting trending tags: {str(e)}")
        return pd.DataFrame()

# Function to extract posts for trending tags
def extract_posts_for_tags(tags_df, posts_per_tag=10):
    """Extract posts for each trending tag"""
    all_posts = []

    # Get top 20 tags by total score
    top_tags = tags_df.sort_values(by='total_score', ascending=False).head(20)['tag'].tolist()

    print(f"Extracting posts for top {len(top_tags)} trending tags...")

    for tag in top_tags:
        print(f"Processing tag: #{tag}")

        try:
            # Use the search method with proper parameters
            # The search_v2 method signature has changed, so we're checking API docs
            # Try different search methods
            try:
                # Try first with the regular search method (which is more stable)
                results = mastodon.search(q=f"#{tag}", result_type="statuses")
                statuses = results.get('statuses', [])
            except Exception as search_error:
                print(f"Error with regular search: {search_error}")
                try:
                    # Fall back to search_v2 if available but try without parameters first
                    results = mastodon.search_v2(f"#{tag}")

                    # Try to extract statuses from the results
                    if hasattr(results, 'statuses'):
                        statuses = results.statuses
                    elif isinstance(results, dict) and 'statuses' in results:
                        statuses = results['statuses']
                    else:
                        # Try to understand the structure
                        print(f"Unexpected results structure for #{tag}")
                        if isinstance(results, dict):
                            print(f"Available keys: {results.keys()}")
                        statuses = []
                except Exception as search_v2_error:
                    print(f"Error with search_v2: {search_v2_error}")
                    statuses = []

            # Limit the number of statuses we process
            for toot in statuses[:posts_per_tag]:
                # Clean HTML content
                content = BeautifulSoup(toot["content"], "html.parser").get_text()

                post = {
                    "tag": tag,
                    "content": content,
                    "created_at": toot["created_at"],
                    "favourites_count": toot["favourites_count"],
                    "reblogs_count": toot["reblogs_count"],
                    "replies_count": toot["replies_count"],
                    "language": toot.get("language", "unknown"),
                    "post_id": toot["id"],
                    "username": toot["account"]["username"],
                    "followers_count": toot["account"]["followers_count"]
                }
                all_posts.append(post)

            time.sleep(1)  # Respect API rate limits

        except Exception as e:
            print(f"Error extracting posts for tag #{tag}: {str(e)}")

    posts_df = pd.DataFrame(all_posts)
    print(f"Extracted {len(posts_df)} posts across {len(top_tags)} trending tags")

    return posts_df

# Function for text preprocessing
def preprocess_text(text):
    """Clean and preprocess text for analysis"""
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)

    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags symbols (keep the words)
    text = re.sub(r'#', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]

    return ' '.join(filtered_tokens)

# Main execution
def run_trend_analysis():
    """Run the full trend analysis process"""
    print("Starting Mastodon trend analysis...")

    # 1. Extract trending tags
    trends_df = extract_trending_tags()

    if trends_df.empty:
        print("Failed to extract trending tags. Exiting.")
        return

    # Save trending tags to CSV
    trends_csv = f"mastodon_trending_tags_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    trends_df.to_csv(trends_csv, index=False)
    print(f"Trending tags saved to {trends_csv}")

    # 2. Extract posts for top trending tags
    posts_df = extract_posts_for_tags(trends_df)

    if not posts_df.empty:
        # Save posts to CSV
        posts_csv = f"mastodon_trending_posts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        posts_df.to_csv(posts_csv, index=False)
        print(f"Trending posts saved to {posts_csv}")

    print("Trend tags extraction completed!")
    return trends_df

if __name__ == "__main__":
    trends_df = run_trend_analysis()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Starting Mastodon trend analysis...
Extracting trending hashtags from Mastodon...
Extracted 10 trending tags
Trending tags saved to mastodon_trending_tags_20250416_112033.csv
Extracting posts for top 10 trending tags...
Processing tag: #springfeverasongorpoem
Error with regular search: ('Mastodon API returned error', 403, 'Forbidden', 'This action is outside the authorized scopes')
Error with search_v2: ('Mastodon API returned error', 403, 'Forbidden', 'This action is outside the authorized scopes')
Processing tag: #complaintsongsorpoems
Error with regular search: ('Mastodon API returned error', 403, 'Forbidden', 'This action is outside the authorized scopes')
Error with search_v2: ('Mastodon API returned error', 403, 'Forbidden', 'This action is outside the authorized scopes')
Processing tag: #cve
Error with regular search: ('Mastodon API returned error', 403, 'Forbidden', 'This action is outside the authorized scopes')
Error with search_v2: ('Mastodon API returned error', 403, 'Forbi

In [10]:
# Add a timestamp to the trend data
trends_df['datetime_collected'] = datetime.utcnow()

In [11]:
# remove history column from the dataset
trends_df = trends_df.drop(columns=['history'])
trends_df

,tag,url,uses_today,accounts_today,uses_week,total_score,datetime_collected
0,complaintsongsorpoems,https://mastodon.social/tags/complaintsongsorp...,0,0,241,241,2025-04-16 11:20:47.154306
1,MeerMittwoch,https://mastodon.social/tags/meermittwoch,0,0,61,61,2025-04-16 11:20:47.154306
2,自介,https://mastodon.social/tags/%E8%87%AA%E4%BB%8B,0,0,80,80,2025-04-16 11:20:47.154306
3,TuneTuesday,https://mastodon.social/tags/tunetuesday,0,0,84,84,2025-04-16 11:20:47.154306
4,4chan,https://mastodon.social/tags/4chan,0,0,97,97,2025-04-16 11:20:47.154306
5,MardiPatisserie,https://mastodon.social/tags/MardiPatisserie,0,0,56,56,2025-04-16 11:20:47.154306
6,memecredi,https://mastodon.social/tags/memecredi,0,0,29,29,2025-04-16 11:20:47.154306
7,cve,https://mastodon.social/tags/cve,0,0,237,237,2025-04-16 11:20:47.154306
8,tercinema,https://mastodon.social/tags/tercinema,0,0,68,68,2025-04-16 11:20:47.154306
9,springfeverasongorpoem,https://mastodon.social/tags/springfeverasongo...,0,0,266,266,2025-04-16 11:20:47.154306


In [12]:
# Convert to dictionary and save
trend_records = trends_df.to_dict(orient='records')

## Save Trending Tags to MongoDB

In [13]:
db.trend_collection.insert_many(trend_records)
print(f"✅ Inserted {len(trend_records)} trending tags into MongoDB.")

✅ Inserted 10 trending tags into MongoDB.


# Get post comments at Mastodon

In [14]:
# Load trending first 10 tags
tags = [doc["tag"] for doc in db.trend_collection.find().limit(20)]
# tags = [doc["tag"] for doc in trend_collection.find().sort("created_at", 1).limit(10)]
print("Trending Tags:", tags)

Trending Tags: ['springfeverasongorpoem', 'Fotomontag', 'PhotoMonday', 'krull', 'singlepeopleslife', 'blueorigin', 'MonochromeMonday', 'TuneTuesday', 'WorldQuantumDay', 'MondayMotivation', 'complaintsongsorpoems', 'MeerMittwoch', '自介', 'TuneTuesday', '4chan', 'MardiPatisserie', 'memecredi', 'cve', 'tercinema', 'springfeverasongorpoem']


In [16]:
from datetime import datetime

for tag in tags:
    print(f"📌 Searching posts for #{tag}")
    posts = mastodon.timeline_hashtag(tag, limit=10)

    for post in posts:
        post_id = post['id']
        post_url = post['url']

        # Fetch comments (replies) to this post
        context = mastodon.status_context(post_id)
        comments = context['descendants']

        for comment in comments:
            comment_text = comment['content']
            created_at = comment['created_at']

            # Clean HTML tags (Mastodon posts are in HTML)
            from bs4 import BeautifulSoup
            plain_text = BeautifulSoup(comment_text, "html.parser").get_text()

            # Prepare for sentiment analysis
            doc = {
                "type": "comment",
                "source": "mastodon",
                "tag": f"#{tag}",
                "text": plain_text,
                "created_at": created_at,
                "post_url": post_url
            }
            sentiment_collection.insert_one(doc)


📌 Searching posts for #springfeverasongorpoem
📌 Searching posts for #Fotomontag
📌 Searching posts for #PhotoMonday
📌 Searching posts for #krull
📌 Searching posts for #singlepeopleslife
📌 Searching posts for #blueorigin
📌 Searching posts for #MonochromeMonday
📌 Searching posts for #TuneTuesday
📌 Searching posts for #WorldQuantumDay
📌 Searching posts for #MondayMotivation
📌 Searching posts for #complaintsongsorpoems
📌 Searching posts for #MeerMittwoch
📌 Searching posts for #自介
📌 Searching posts for #TuneTuesday
📌 Searching posts for #4chan
📌 Searching posts for #MardiPatisserie
📌 Searching posts for #memecredi
📌 Searching posts for #cve
📌 Searching posts for #tercinema
📌 Searching posts for #springfeverasongorpoem
